<a href="https://colab.research.google.com/github/Dreadnought73/AI_projects/blob/main/News_Summarizer_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using prompt template

**News summarizer with DDGS and OpenAI**

In this section, with the help of an LLM, it uses web search to find relevant articles and summarizes them based on the prompt template.

In [5]:
pip install duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.1 MB/s eta 0:00:00


In [49]:
from duckduckgo_search import DDGS
from openai import OpenAI

llm = OpenAI(api_key='your_api_key')

PROMPT_TEMPLATE = """
You are an expert in synthesizing and summarizing news articles across a wide range of topics.
Your output should reflect both deep analytical ability and journalistic clarity, combining the precision of AP/Reuters style with the brevity and relevance expected in the digital age.

Your task:
1. Content Analysis & Synthesis
   - Carefully read and analyze the provided news articles.
   - Identify the key themes, facts, and developments.
   - Distill overlapping or contrasting perspectives into a unified understanding.
   - Create a concise, insightful report that captures the essence of the topic without losing important nuance.

2. Summary Format & Style Guidelines
   - Begin with the most newsworthy development.
   - Highlight the key stakeholders, actions taken, and critical data or numbers.
   - Explain why the development is significant now**, including any immediate implications or broader impact.
   - Follow this structure:
     [Major News] + [Key Details/Data] + [Why It Matters/What’s Next]
   - Write a single paragraph (300–400 words).
   - Style should include:
     - Strong, active verbs
     - Specificity and clarity
     - Objective tone and neutral framing
     - Definitions for technical terms if necessary
     - No fluff—make every word count

3. Final Section: Sources
   - At the end of the summary, provide a Markdown list of the sources used (URLs, publication names, or article titles).

Your output should both inform and engage while maintaining high editorial standards.
"""

# Using DuckDuckGo, it searches the news for 7 relevant articles
def search_news(query: str):
    with DDGS() as ddgs:
        results = ddgs.news(query, max_results=7)
        if results:
          news_results = '/n/n'.join([
              f"Summary: {result.get('body')}/nSource: {result.get('source')}"
          for result in results])
          return news_results
        else:
            return f"No news found for {topic}."

# Based on one prompt, it creates the summaries
def summarize_content(content:str):
  response = llm.chat.completions.create(
      model='gpt-4o-mini',
      messages=[
          {"role": "developer", "content": PROMPT_TEMPLATE},
          {"role": "user", "content": content}
      ]
  )
  return response.choices[0].message.content

In [50]:
# Combines the previous functions
def router(topic: str):
  raw_news = search_news(topic)
  summary_response = summarize_content(raw_news)
  return summary_response

# Using multi-agent system

In [ ]:
!pip install git+https://github.com/openai/swarm.git

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-e_pe85o0
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-e_pe85o0
  Resolved https://github.com/openai/swarm.git to commit 0c82d7d868bb8e2d380dfd2a319b5c3a1f4c0cb9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

**News summarizer with Swarm**

In this section, with the help of an LLM, the task is divided among 3 separate agents. One searches the news articles, one extracts the information that can be useful, one summarizes the results.

In [ ]:
from duckduckgo_search import DDGS
from swarm import Swarm, Agent
from openai import OpenAI

In [ ]:
llm = OpenAI(api_key='your_api_key')
client = Swarm(client=llm)

# Using DDGS to build a function that searches the top 5 results for the given topic.
# If contents found, it combines them.
def search_news(topic: str):
    with DDGS() as ddgs:
        results = ddgs.text(topic, max_results=5)
        if results:
            new_results = '/n/n'.join([
                f"Title: {result['title']}\nSummary: {result['body']}"
                for result in results
            ])
        else:
            return f"No news found for {topic}."

search_agent = Agent(
    name="News Searcher",
    model='gpt-4o-mini',
    instructions= """
    You are an expert at searching for news articles.
    Your task is to find the latest news articles related to a given topic.
    Make sure to use reliable and reputable sources during your search.
    Return the raw search result in a structured format.
    """,
    functions=[search_news]
)

In [ ]:
synthesis_agent = Agent(
    name='Synthesiser Agent',
    model='gpt-4o-mini',
    instructions="""
    You are an expert at synthesising news articles on a wide range of topics.
    Your task is to:
    1. Analyze the raw news articles provided
    2. Identify the key themes and important information
    3. Combine information from multiple sources
    4. Create a comprehensive but concise synthesis
    5. Focus on facts and maintain journalistic objectivity
    6. Write in a clear, professional style
    Provide a 2-3 paragraph synthesis of the main points.
    """
)

In [ ]:
summary_agent = Agent(
    name='Summary Agent',
    model='gpt-4o-mini',
    instructions= """
    You are an expert news summarizer combining AP and Reuters style clarity with digital-age brevity.

    Your task:
    1. Core Information:
       - Lead with the most newsworthy development
       - Include key stakeholders and their actions
       - Add critical numbers/data if relevant
       - Explain why this matters now
       - Mention immediate implications

    2. Style Guidelines:
       - Use strong, active verbs
       - Be specific, not general
       - Maintain journalistic objectivity
       - Make every word count
       - Explain technical terms if necessary

    Format: Create a single paragraph of 250-400 words that informs and engages.
    Pattern: [Major News] + [Key Details/Data] + [Why It Matters/What's Next]

    Focus on answering: What happened? Why is it significant? What's the impact?

    IMPORTANT: Provide ONLY the summary paragraph. Avoid including any introductory phrases,
    labels, or meta-text like "Here's a summary" or "In AP/Reuters style."
    Start directly with the news content.
    """
)

In [ ]:
# Combines the 3 Agents under one workflow
def process_news(topic: str):
  """Processing the whole workflow"""
  with st.status('Processing information...', expanded=True) as status:
    st.write('Searching for news articles...')
    search_response = client.run(
        agent=search_agent,
        messages=[{"role": "user", "content": f"Find recent news about {topic}"}]
    )
    raw_news = search_response.messages[-1]['content']

    st.write('Synthesizing news articles...')
    synthesis_response = client.run(
        agent=synthesis_agent,
        messages=[{"role": "user", "content": raw_news}]
    )
    synthesised_news = synthesis_response.messages[-1]['content']

    st.write('Summarizing news articles...')
    summary_response = client.run(
        agent=summary_agent,
        messages=[{"role": "user", "content": synthesised_news}]
    )

  return raw_news, synthesised_news, summary_response.messages[-1]['content']